### KAS 조업사 예측

In [ ]:
import pandas as pd
import random

df = pd.read_csv("kas_queue.csv")
df = pd.read_csv("VTT.csv")

import pandas as pd

def time_to_minutes(t):
    try:
        if isinstance(t, float):
            return int(round(t * 24 * 60))
        h, m = map(int, str(t).split(":"))
        return h * 60 + m
    except:
        return 20

def minutes_to_time(m):
    return f"{m // 60:02}:{m % 60:02}"

def get_worktime(code):
    return {'C': 20, 'E': 25, 'F': 30}.get(code, 20)

kas_df = pd.read_csv("kas_queue.csv")
vtt_df = pd.read_csv("VTT.csv")

kas_df['SOBT_MIN'] = kas_df['SOBT'].apply(time_to_minutes)
kas_df = kas_df.sort_values('SOBT_MIN').reset_index(drop=True)

zone_list = []
group_size = 6
for i in range(len(kas_df)):
    stn = str(kas_df.loc[i, 'STN'])
    code = str(kas_df.loc[i, 'CODE'])
    if stn.isdigit() and len(stn) == 3 and stn.startswith("6"):
        zone_list.append("CD")
    elif code == 'F':
        zone_list.append("CD")
    else:
        zone_list.append("TC" if i % group_size < 4 else "CD")
kas_df['ZONE'] = zone_list

eobt_list, epot_temp = [], []
for i in range(len(kas_df)):
    if i < group_size:
        eobt = kas_df.loc[i, 'SOBT_MIN']
    else:
        eobt = epot_temp[i - group_size]
    eobt_list.append(eobt)
    epot_temp.append(eobt + 40)
kas_df['EOBT_MIN'] = eobt_list
kas_df['EOBT'] = kas_df['EOBT_MIN'].apply(minutes_to_time)

kas_df['STN'] = kas_df['STN'].astype(str)
vtt_df['STN'] = vtt_df['STN'].astype(str)
vtt_df['VTT_MIN'] = vtt_df['VTT'].apply(time_to_minutes)

epit_list = []
for i in range(len(kas_df)):
    stn = kas_df.loc[i, 'STN']
    zone = kas_df.loc[i, 'ZONE']
    eobt = kas_df.loc[i, 'EOBT_MIN']

    match_row = vtt_df[(vtt_df['STN'] == stn) & (vtt_df['ZONE'] == zone)]
    vtt_time = int(match_row['VTT_MIN'].values[0]) if not match_row.empty else 20
    epit_list.append(eobt + vtt_time)

kas_df['EPIT_MIN'] = epit_list
kas_df['EPIT'] = kas_df['EPIT_MIN'].apply(minutes_to_time)

epot_list = []
for i in range(len(kas_df)):
    epit = kas_df.loc[i, 'EPIT_MIN']
    code = kas_df.loc[i, 'CODE']
    work = get_worktime(code)
    epot_list.append(epit + work)
kas_df['EPOT_MIN'] = epot_list
kas_df['EPOT'] = kas_df['EPOT_MIN'].apply(minutes_to_time)
df_kas = kas_df

final_df = kas_df[['FLIGHT', 'CODE', 'STN', 'SOBT', 'deicer', 'ZONE', 'EOBT', 'EPIT', 'EPOT']]
from IPython.display import display
display(final_df)

,FLIGHT,CODE,STN,SOBT,deicer,ZONE,EOBT,EPIT,EPOT
0,JNA261,E,266,7:15,KAS,TC,07:15,07:33,07:58
1,KAL787,E,261R,7:25,KAS,TC,07:25,07:40,08:05
2,JNA201,E,256,7:25,KAS,TC,07:25,07:33,07:58
3,KAL771,C,250,7:30,KAS,TC,07:30,07:41,08:01
4,JNA345,E,258R,7:30,KAS,CD,07:30,08:02,08:27
...,...,...,...,...,...,...,...,...,...
81,TAX701,E,112,11:35,KAS,TC,16:10,16:33,16:58
82,AMX091,E,257,11:40,KAS,CD,16:10,16:35,17:00
83,KAL925,E,242,11:55,KAS,CD,16:20,16:47,17:12
84,CAL165,C,252,12:05,KAS,TC,16:35,16:54,17:14


AAP 조업사 예측

In [ ]:
import pandas as pd
import random

def time_to_minutes(t):
    try:
        if isinstance(t, float):
            return int(round(t * 24 * 60))
        h, m = map(int, str(t).split(":"))
        return h * 60 + m
    except:
        return 20

def minutes_to_time(m):
    return f"{m // 60:02}:{m % 60:02}"

def get_worktime(code):
    return {'C': 20, 'E': 25, 'F': 30}.get(code, 20)

aap_df = pd.read_csv("aap_queue.csv")
vtt_df = pd.read_csv("VTT.csv")

aap_df['SOBT_MIN'] = aap_df['SOBT'].apply(time_to_minutes)
aap_df = aap_df.sort_values('SOBT_MIN').reset_index(drop=True)

zone_list = []
for i, row in aap_df.iterrows():
    stn_str = str(row['STN'])
    if stn_str.isdigit() and len(stn_str) == 3 and stn_str.startswith("6"):
        zone_list.append("CD")
    else:
        zone_list.append(random.choice(["AS", "CD"]))
aap_df['ZONE'] = zone_list

aap_df['STN'] = aap_df['STN'].astype(str)
vtt_df['STN'] = vtt_df['STN'].astype(str)
vtt_df['VTT_MIN'] = vtt_df['VTT'].apply(time_to_minutes)

queues = [[] for _ in range(5)]

eobt_list = []
epit_list = []
epot_list = []

for i, row in aap_df.iterrows():
    if all(queues):
        earliest_queue_idx = min(range(5), key=lambda x: queues[x][-1])
    else:
        earliest_queue_idx = queues.index([])

    eobt = max(row['SOBT_MIN'], queues[earliest_queue_idx][-1] if queues[earliest_queue_idx] else row['SOBT_MIN'])

    stn = row['STN']
    zone = row['ZONE']
    match = vtt_df[(vtt_df['STN'] == stn) & (vtt_df['ZONE'] == zone)]
    vtt_min = int(match['VTT_MIN'].values[0]) if not match.empty else 20
    epit = eobt + vtt_min

    worktime = get_worktime(row['CODE'])
    epot = epit + worktime

    queues[earliest_queue_idx].append(epot)

    eobt_list.append(eobt)
    epit_list.append(epit)
    epot_list.append(epot)

aap_df['EOBT_MIN'] = eobt_list
aap_df['EOBT'] = aap_df['EOBT_MIN'].apply(minutes_to_time)
aap_df['EPIT_MIN'] = epit_list
aap_df['EPIT'] = aap_df['EPIT_MIN'].apply(minutes_to_time)
aap_df['EPOT_MIN'] = epot_list
aap_df['EPOT'] = aap_df['EPOT_MIN'].apply(minutes_to_time)
df_aap = aap_df

final_df = aap_df[['FLIGHT', 'CODE', 'STN', 'SOBT', 'deicer', 'ZONE', 'EOBT', 'EPIT', 'EPOT']]
from IPython.display import display
display(final_df)

,FLIGHT,CODE,STN,SOBT,deicer,ZONE,EOBT,EPIT,EPOT
0,ABL164,C,47,7:15,AAP,AS,07:15,07:45,08:05
1,ASV711,C,41,7:15,AAP,CD,07:15,07:40,08:00
2,ASV727,C,18,7:20,AAP,CD,07:20,07:43,08:03
3,AAR731,E,15,7:30,AAP,AS,07:30,07:53,08:18
4,CES5050,C,131,7:45,AAP,AS,07:45,08:14,08:34
5,AMU825,C,45,7:50,AAP,CD,08:00,08:29,08:49
6,AAR701,E,23,7:50,AAP,AS,08:03,08:31,08:56
7,AAR112,E,7,7:55,AAP,AS,08:05,08:27,08:52
8,ASV705,C,117,8:10,AAP,AS,08:18,08:45,09:05
9,AAR122,E,11,8:15,AAP,AS,08:34,08:57,09:22


### SHP 조업사 예측

In [ ]:
import pandas as pd

def time_to_minutes(t):
    try:
        if isinstance(t, float):
            return int(round(t * 24 * 60))
        h, m = map(int, str(t).split(":"))
        return h * 60 + m
    except:
        return 20

def minutes_to_time(m):
    return f"{m // 60:02}:{m % 60:02}"

def get_worktime(code):
    return {'C': 20, 'E': 25, 'F': 30}.get(code, 20)

shp_df = pd.read_csv("shp_queue.csv")
vtt_df = pd.read_csv("VTT.csv")

shp_df['SOBT_MIN'] = shp_df['SOBT'].apply(time_to_minutes)
shp_df = shp_df.sort_values('SOBT_MIN').reset_index(drop=True)

shp_df['ZONE'] = "CD"

shp_df['STN'] = shp_df['STN'].astype(str)
vtt_df['STN'] = vtt_df['STN'].astype(str)
vtt_df['VTT_MIN'] = vtt_df['VTT'].apply(time_to_minutes)

queues = [[] for _ in range(3)]
eobt_list, epit_list, epot_list = [], [], []

for i, row in shp_df.iterrows():
    if all(queues):
        earliest_queue_idx = min(range(3), key=lambda x: queues[x][-1])
    else:
        earliest_queue_idx = queues.index([])

    eobt = max(row['SOBT_MIN'], queues[earliest_queue_idx][-1] if queues[earliest_queue_idx] else row['SOBT_MIN'])

    stn = row['STN']
    zone = row['ZONE']
    match = vtt_df[(vtt_df['STN'] == stn) & (vtt_df['ZONE'] == zone)]
    vtt_time = int(match['VTT_MIN'].values[0]) if not match.empty else 20
    epit = eobt + vtt_time

    worktime = get_worktime(row['CODE'])
    epot = epit + worktime

    queues[earliest_queue_idx].append(epot)

    eobt_list.append(eobt)
    epit_list.append(epit)
    epot_list.append(epot)

shp_df['EOBT_MIN'] = eobt_list
shp_df['EOBT'] = shp_df['EOBT_MIN'].apply(minutes_to_time)
shp_df['EPIT_MIN'] = epit_list
shp_df['EPIT'] = shp_df['EPIT_MIN'].apply(minutes_to_time)
shp_df['EPOT_MIN'] = epot_list
shp_df['EPOT'] = shp_df['EPOT_MIN'].apply(minutes_to_time)
df_shp = shp_df

final_df = shp_df[['FLIGHT', 'CODE', 'STN', 'SOBT', 'deicer', 'ZONE', 'EOBT', 'EPIT', 'EPOT']]
from IPython.display import display
display(final_df)

,FLIGHT,CODE,STN,SOBT,deicer,ZONE,EOBT,EPIT,EPOT
0,GTI7612,E,683,7:15,SHP,CD,07:15,07:35,08:00
1,APJ712,C,106,7:30,SHP,CD,07:30,07:48,08:08
2,ESR671,C,114,7:35,SHP,CD,07:35,07:55,08:15
3,ESR621,C,108,7:40,SHP,CD,08:00,08:20,08:40
4,TWB611,C,126,7:45,SHP,CD,08:08,08:31,08:51
5,TWB171,C,102,7:55,SHP,CD,08:15,08:37,08:57
6,TWB287,C,118,7:55,SHP,CD,08:40,09:05,09:25
7,TWB301,C,26,8:00,SHP,CD,08:51,09:31,09:51
8,ESR601,C,130,8:00,SHP,CD,08:57,09:16,09:36
9,TWB613,C,127,8:05,SHP,CD,09:25,09:47,10:07


### SPI 조업사 예측

In [ ]:
import pandas as pd

def time_to_minutes(t):
    try:
        if isinstance(t, float):
            return int(round(t * 24 * 60))
        h, m = map(int, str(t).split(":"))
        return h * 60 + m
    except:
        return 20

def minutes_to_time(m):
    return f"{m // 60:02}:{m % 60:02}"

def get_worktime(code):
    return {'C': 20, 'E': 25, 'F': 30}.get(code, 20)

spi_df = pd.read_csv("spi_queue.csv")
vtt_df = pd.read_csv("VTT.csv")

spi_df['SOBT_MIN'] = spi_df['SOBT'].apply(time_to_minutes)
spi_df = spi_df.sort_values('SOBT_MIN').reset_index(drop=True)

spi_df['ZONE'] = "MS"

spi_df['STN'] = spi_df['STN'].astype(str)
vtt_df['STN'] = vtt_df['STN'].astype(str)
vtt_df['VTT_MIN'] = vtt_df['VTT'].apply(time_to_minutes)

queues = [[] for _ in range(3)]
eobt_list, epit_list, epot_list = [], [], []

for i, row in spi_df.iterrows():
    if all(queues):
        earliest_queue_idx = min(range(3), key=lambda x: queues[x][-1])
    else:
        earliest_queue_idx = queues.index([])

    eobt = max(row['SOBT_MIN'], queues[earliest_queue_idx][-1] if queues[earliest_queue_idx] else row['SOBT_MIN'])

    stn = row['STN']
    zone = row['ZONE']
    match = vtt_df[(vtt_df['STN'] == stn) & (vtt_df['ZONE'] == zone)]
    vtt_time = int(match['VTT_MIN'].values[0]) if not match.empty else 20
    epit = eobt + vtt_time

    worktime = get_worktime(row['CODE'])
    epot = epit + worktime

    queues[earliest_queue_idx].append(epot)

    eobt_list.append(eobt)
    epit_list.append(epit)
    epot_list.append(epot)

spi_df['EOBT_MIN'] = eobt_list
spi_df['EOBT'] = spi_df['EOBT_MIN'].apply(minutes_to_time)
spi_df['EPIT_MIN'] = epit_list
spi_df['EPIT'] = spi_df['EPIT_MIN'].apply(minutes_to_time)
spi_df['EPOT_MIN'] = epot_list
spi_df['EPOT'] = spi_df['EPOT_MIN'].apply(minutes_to_time)
df_spi = spi_df

final_df = spi_df[['FLIGHT', 'CODE', 'STN', 'SOBT', 'deicer', 'ZONE', 'EOBT', 'EPIT', 'EPOT']]
from IPython.display import display
display(final_df)

,FLIGHT,CODE,STN,SOBT,deicer,ZONE,EOBT,EPIT,EPOT
0,VJC879,C,129,7:00,SPI,MS,07:00,07:19,07:39
1,PAL467,C,48,8:00,SPI,MS,08:00,08:14,08:34
2,CPA417,E,48,10:10,SPI,MS,10:10,10:24,10:49
3,LOT098,E,21,11:20,SPI,MS,11:20,11:47,12:12
4,DLH719,E,41,11:40,SPI,MS,11:40,11:59,12:24
5,VJC863,C,117,11:40,SPI,MS,11:40,12:09,12:29
6,CSS280,C,628,11:55,SPI,MS,12:12,12:43,13:03


### JAS 조업사 예측

In [ ]:
jas_df = pd.read_csv("jas_queue.csv")
vtt_df = pd.read_csv("VTT.csv")

jas_df['SOBT_MIN'] = jas_df['SOBT'].apply(time_to_minutes)
jas_df = jas_df.sort_values('SOBT_MIN').reset_index(drop=True)

jas_df['ZONE'] = "CD"

jas_df['STN'] = jas_df['STN'].astype(str)
vtt_df['STN'] = vtt_df['STN'].astype(str)
vtt_df['VTT_MIN'] = vtt_df['VTT'].apply(time_to_minutes)

queues = [[] for _ in range(2)]
eobt_list, epit_list, epot_list = [], [], []

for i, row in jas_df.iterrows():
    if all(queues):
        earliest_queue_idx = min(range(2), key=lambda x: queues[x][-1])
    else:
        earliest_queue_idx = queues.index([])

    eobt = max(row['SOBT_MIN'], queues[earliest_queue_idx][-1] if queues[earliest_queue_idx] else row['SOBT_MIN'])

    stn = row['STN']
    zone = row['ZONE']
    match = vtt_df[(vtt_df['STN'] == stn) & (vtt_df['ZONE'] == zone)]
    vtt_time = int(match['VTT_MIN'].values[0]) if not match.empty else 20
    epit = eobt + vtt_time

    worktime = get_worktime(row['CODE'])
    epot = epit + worktime

    queues[earliest_queue_idx].append(epot)

    eobt_list.append(eobt)
    epit_list.append(epit)
    epot_list.append(epot)

jas_df['EOBT_MIN'] = eobt_list
jas_df['EOBT'] = jas_df['EOBT_MIN'].apply(minutes_to_time)
jas_df['EPIT_MIN'] = epit_list
jas_df['EPIT'] = jas_df['EPIT_MIN'].apply(minutes_to_time)
jas_df['EPOT_MIN'] = epot_list
jas_df['EPOT'] = jas_df['EPOT_MIN'].apply(minutes_to_time)
df_jas = jas_df

final_df = jas_df[['FLIGHT', 'CODE', 'STN', 'SOBT', 'deicer', 'ZONE', 'EOBT', 'EPIT', 'EPOT']]
from IPython.display import display
display(final_df)

,FLIGHT,CODE,STN,SOBT,deicer,ZONE,EOBT,EPIT,EPOT
0,JJA1701,C,112,7:00,JAS,CD,07:00,07:18,07:38
1,JJA1301,C,122,7:10,JAS,CD,07:10,07:28,07:48
2,JJA1501,C,37,7:20,JAS,CD,07:38,08:06,08:26
3,JJA1611,C,38,7:25,JAS,CD,07:48,08:11,08:31
4,JJA8401,C,30,8:20,JAS,CD,08:26,09:00,09:20
5,JJA1101,C,112,8:30,JAS,CD,08:31,08:49,09:09
6,JJA6101,C,41,9:05,JAS,CD,09:09,09:34,09:54
7,JJA1303,C,128,9:10,JAS,CD,09:20,09:42,10:02
8,JJA2121,C,28,9:10,JAS,CD,09:54,10:27,10:47
9,JJA3211,C,101,9:30,JAS,CD,10:02,10:26,10:46


### 5개 조업사 통합 지연시간 예측

In [ ]:
import pandas as pd

dataframes = [df_jas, df_spi, df_shp, df_aap, df_kas]
columns_to_merge = ['FLIGHT', 'STN', 'SOBT', 'EOBT']
cleaned_dfs = []

for df in dataframes:
    df.columns = df.columns.str.strip()
    if 'Unnamed: 0' in df.columns:
        df.drop(columns=['Unnamed: 0'], inplace=True)
    for col in columns_to_merge:
        if col not in df.columns:
            df[col] = None
    cleaned_dfs.append(df[columns_to_merge])

merged_df = pd.concat(cleaned_dfs, ignore_index=True)

merged_df['EOBT'] = merged_df.apply(
    lambda row: row['SOBT'] if pd.isna(row['EOBT']) else row['EOBT'],
    axis=1
)

def time_to_minutes(t):
    try:
        h, m = map(int, str(t).strip().split(":"))
        return h * 60 + m
    except:
        return float('inf')

merged_df['SOBT_MIN'] = merged_df['SOBT'].apply(time_to_minutes)
merged_df = merged_df.sort_values(by='SOBT_MIN').drop(columns=['SOBT_MIN']).reset_index(drop=True)

from IPython.display import display
display(merged_df)

,FLIGHT,STN,SOBT,EOBT
0,JJA1701,112,7:00,07:00
1,VJC879,129,7:00,07:00
2,JJA1301,122,7:10,07:10
3,GTI7612,683,7:15,07:15
4,ASV711,41,7:15,07:15
...,...,...,...,...
196,AAR154,22,12:10,16:14
197,CDG4610,35,12:15,16:33
198,AAR339,11,12:15,16:32
199,AIC313,31,12:15,14:15


### 도착편 Gate Occupied 예측

In [ ]:
import pandas as pd
from collections import defaultdict, deque

arr_file_path = "arr.csv"
dep_file_path = "merged_eobt_data.csv"

arr_df = pd.read_csv(arr_file_path)
merged_df = pd.read_csv(dep_file_path)

def time_to_minutes(t):
    try:
        h, m = map(int, str(t).strip().split(":"))
        return h * 60 + m
    except:
        return None

def minutes_to_time(m):
    return f"{int(m // 60):02}:{int(m % 60):02}"

arr_df['EIBT_MIN'] = arr_df['EIBT'].apply(time_to_minutes)
merged_df['SOBT_MIN'] = merged_df['SOBT'].apply(time_to_minutes)
merged_df['EOBT_MIN'] = merged_df['EOBT'].apply(time_to_minutes)

arr_df['FLIGHT'] = arr_df['FLIGHT'].astype(str).str.strip()
arr_df['LKF'] = arr_df['LKF'].astype(str).str.strip()
merged_df['FLIGHT'] = merged_df['FLIGHT'].astype(str).str.strip()

dep_flights = set(merged_df['FLIGHT'].unique())
arr_df['LINKED'] = arr_df['LKF'].apply(lambda x: x in dep_flights)

start_min = 7 * 60
end_min = 14 * 60
arrivals = arr_df[
    (arr_df['EIBT_MIN'] >= start_min) &
    (arr_df['EIBT_MIN'] <= end_min) &
    (~arr_df['LINKED'])
].copy()

timeline = []
for _, row in merged_df.iterrows():
    stn = str(row['STN'])
    sobt = row['SOBT_MIN']
    eobt = row['EOBT_MIN']
    flight = row['FLIGHT']
    if pd.notnull(sobt):
        timeline.append((sobt, 'enqueue', stn, flight))
    if pd.notnull(eobt):
        timeline.append((eobt, 'dequeue', stn, flight))

for _, row in arrivals.iterrows():
    stn = str(row['STN'])
    eibt = row['EIBT_MIN']
    flight = row['FLIGHT']
    timeline.append((eibt, 'arrive', stn, flight))

queue = defaultdict(deque)
occupied_arrivals = []

for t, action, stn, flight in sorted(timeline):
    t_str = minutes_to_time(t)
    if action == 'enqueue':
        queue[stn].append(flight)
    elif action == 'dequeue':
        if queue[stn]:
            queue[stn].popleft()
    elif action == 'arrive':
        if queue[stn]:
            occupied_arrivals.append({'TIME': t_str, 'STN': stn, 'FLIGHT': flight, 'STATUS': 'OCCUPIED'})
        else:
            occupied_arrivals.append({'TIME': t_str, 'STN': stn, 'FLIGHT': flight, 'STATUS': 'AVAILABLE'})

result_df = pd.DataFrame(occupied_arrivals)
result_df['TIME_MIN'] = result_df['TIME'].apply(time_to_minutes)

latest_eobt_per_stn = (
    merged_df[['STN', 'EOBT_MIN']]
    .dropna()
    .groupby('STN')['EOBT_MIN']
    .max()
    .reset_index()
    .rename(columns={'EOBT_MIN': 'LATEST_EOBT_MIN'})
)

occupied_only = result_df[result_df['STATUS'] == 'OCCUPIED'].copy()
occupied_only = pd.merge(
    occupied_only,
    latest_eobt_per_stn,
    on='STN',
    how='left'
)

occupied_only['LATEST_EOBT'] = occupied_only['LATEST_EOBT_MIN'].apply(minutes_to_time)
occupied_only['ARR_TIME'] = occupied_only['TIME_MIN'].apply(minutes_to_time)
occupied_only['DURATION_MIN'] = occupied_only['LATEST_EOBT_MIN'] - occupied_only['TIME_MIN']

final_df = occupied_only[['ARR_TIME', 'STN', 'FLIGHT', 'LATEST_EOBT', 'DURATION_MIN']]
final_df = final_df.sort_values(by='ARR_TIME').reset_index(drop=True)

from IPython.display import display
display(final_df)

,ARR_TIME,STN,FLIGHT,LATEST_EOBT,DURATION_MIN
0,09:20,246,KAL630,10:05,45
1,09:50,27,TWB432,10:20,30
2,10:30,20,JJA1402,11:06,36
3,10:45,266,KAL952,12:05,80
4,11:00,18,AAR115,09:32,-88
...,...,...,...,...,...
61,13:55,114,ABL163,07:35,-380
62,13:55,126,CSN337,10:20,-215
63,13:55,252,KAL778,16:35,160
64,14:00,115,ESR612,12:00,-120
